字典树（tier tree）    

**概念：**  

例如查找“hello”的过程是 "h" -> "he" -> "hel" -> "hell" -> "hello", 我们使用boolean标志位来判断以该节点结尾的这条路径是不是一个单词。不管是寻找单词还是寻找前缀，我们都是从上往下去查找。比如我们从根节点去找 hello 的第一个字符，也就是"h"， 我们会直接访问 root.children['h' - 'a'] 所代表的节点。 因为是从上到下的关系，下一层的信息依赖与上一层的children数组，字典树的root不代表任何字符，只是一个搜索的入口。查找一个单词和插入一个单词类似，就是确认一条路径是否存在的过程。 

**两大基本用法：**  
1. 确认一个单词是否在字典中存在   
2. 确认字典中是否含有前缀的单词。  
第二点可以扩展为，求字典中含有某前缀的所有单词，计算字典中含有某前缀的单词的个数，计算字典中含有某前缀出现的频率。  

**性能分析**  

字典树的查找和插入单词的时间复杂度都是 O(L), L是单词的长度。如果是利用哈希表来确认一个字符串是否存在，需要计算元素的哈希值和查找并确认元素是否存在。为了减少哈希冲突，哈希函数需要将整个字符串给扫描一遍，此时的时间复杂度 O(L), 然后根据哈希值找到对应的位置，需要去比较判断当前的字符是不是我们要找的，两个字符串的比较只能是挨个比较，时间负责度也是 O(L)。 因此从时间上来看，在字符串的查找方面，哈希表不会比字典树更优。从空间上来看，由于字典树具有字符串前缀的相关功能，具有相同前缀的单词的存储会被压缩。


字典树在实际中也有被运用到，如「**搜索引擎的自动补全**」, 「**拼写检查**」

In [1]:
# E.g
# 两次insert， 一次插入apple， 一次插入app
# apple {'a': {'p': {'p': {'l': {'e': {'end':True}}}}}}
# app   {'a': {'p': {'p': {'l': {'e': {'end':True}}, 'end': True }}}}

# 实现tier tree，包含insert, search, startwith 三个操作
from collections import defaultdict

class TrieNode:
    def __init__(self):
        self.child = defaultdict(TrieNode)
        self.isword = False

class Trie:
    def __init__(self):
        self.root = TrieNode()
        
    def insert(self, word):
        """
        insert a word into the trie tree
        """
        cur = self.root
        for w in word:
            cur = cur.child[w]
        # 标记终点
        cur.isword = True
        
    def search(self, word):
        """
        return if the word is in the trie
        """
        cur = self.root
        for w in word:
            if w not in cur.child:
                return False
            cur = cur.child[w]
        return cur.isword
    
    def startsWith(self, prefix):
        cur = self.root
        for p in prefix:
            if p not in cur.child:
                return False
            cur = cur.child[p]
        return True
    
tier = Trie()
tier.insert('apple')
print(tier.search('apple'))
print(tier.startsWith('ap'))


True
True


In [2]:
from collections import defaultdict

class TrieNode:
    def __init__(self):
        self.child = defaultdict(TrieNode)
        self.isword = False
        
class Trie:
    def __init__(self):
        self.root = TrieNode()
    
    def insert(self, word):
        cur = self.root
        for w in word:
            cur = cur.child[w]
        cur.isword = True
        
    def search(self, word):
        cur = self.root
        for w in word:
            if w not in cur.child:
                return False
            cur = cur.child[w]
        return cur.isword
    
    def startwith(self, prefix):
        cur = self.root
        for p in prefix:
            if p not in cur.child:
                return False
            cur = cur.child[p]
        return True
            
    
tier = Trie()
tier.insert('apple')
print(tier.search('apple'))
print(tier.startwith('ap'))
            

True
True


1. “.” 表示匹配任意单个字符， “*” 表示匹配0个或多个前面的那一个元素  
2. 递归 + 暴力dfs 求解  
字符串划分为字符，字符串的比较变成字符的比较， s[i, i+1,...n] 是否能匹配 p[j, j+1, j+2...m]的子问题为 s[i+1,...n] 和 p[j+1, j+2...m]  但是字符串s中没有特殊字符，只有字母。字符串p，我们需要考虑两个特殊字符'.'和'*'

In [3]:

# 匹配是指字符串的所有字符匹配整个模式

# s [0, 1, ...,n-1]
# p [0, 1, ...,m-1]
# 要防止数组越界
class Solution:
    
    def match(self, string, i, pattern,  j):
        # 字符串 和 模式串 都到结尾
        if i == len(string)-1 and j == len(pattern)-1:
            return True
        # 模式串到了结尾，字符串没到
        elif j == len(pattern)-1:
            return False    
        # 字符串到了结尾，模式串没到，仍可能成立, 如 a*a*a* 
        # 设置标志位，对"*"的情况讨论
        next = (pattern[j+1] == '*') and ((j+2) < len(pattern))
        # 模式串的第二个字符是 *
        if next:
            # 字符串后移一位，模式串不变，相当于x*匹配多位
            # 字符串不动，模式串后移两位，相当于x*被忽略
            if (string[i] == pattern[j]) or (pattern[j] == '.' and i+1 < len(string)):
                return (self.match(string, i, pattern,  j+2) or self.match(string, i+1, pattern,  j))
            # 字符串第一位和模式串第一位不匹配，模式串第二位为*，模式串后移2位
            else:
                return self.match(string, i, pattern,  j+2)
        # 模式串的第二个字符不是 * 
        else:
            if (string[i] == pattern[j]) or (pattern[j] == '.' and i+1 < len(string)):
                
                return self.match(string, i+1, pattern, j+1)
            else:
                return False

string = 'abc'
pattern = 'a*bc'
s = Solution()
s.match(string, 0, pattern, 0)


True